In [57]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq

from Bio.SeqRecord import SeqRecord


import vcf
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import functools

import numpy as np
import pandas as pd

Entrez.email = "simon.burgermeister@gmail.com"


In [19]:
search_info="S[Gene Name] AND Severe acute respiratory syndrome coronavirus 2[Organism] AND Homo sapiens[Host] AND complete[genome]"

handle = Entrez.esearch(db='nucleotide', term=search_info, usehistory="y", idtype="acc")
search_results = Entrez.read(handle)
handle.close()
print(search_results['Count'])
len_rec=search_results['Count']

acc_list = search_results["IdList"]
count = int(search_results["Count"])
count == len(acc_list)

webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

178834


In [20]:
#count=3

In [60]:
#rettype="fasta"

batch_size = 1
out_handle = open("sequences/samples_test.fasta", "w")
data_array=[]

for start in range(0, count, batch_size):
    end = min(count, start + batch_size)
    print("Going to download record %i to %i" % (start + 1, end))
    fetch_handle = Entrez.efetch(
        db="nucleotide",
        rettype="gb",
        retmode="text",
        retstart=start,
        retmax=batch_size,
        webenv=webenv,
        query_key=query_key,
        idtype="acc",
    )
    
    
    record = SeqIO.read(fetch_handle, "genbank")
    sseq=record.seq
    iid=record.id
    name=record.name
    date=record.annotations['date']
    
    sseqx='>'+name+'\n'+str(sseq)+'\n\n'
    
    for feat in record.features:
        if feat.type=='source':
            col_date=feat.qualifiers['collection_date'][0]
            country=feat.qualifiers['country'][0]
            source=feat.qualifiers['isolation_source'][0]
    
    
    
    
    data_array.append([name, col_date, country, source])
    del name, col_date, country, source
    
    #data = fetch_handle.read()
    fetch_handle.close()
    fetch_handle = Entrez.efetch(
        db="nucleotide",
        rettype="fasta",
        retmode="text",
        retstart=start,
        retmax=batch_size,
        webenv=webenv,
        query_key=query_key,
        idtype="acc",
    )
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
    #out_handle.write(sseqx)

    
out_handle.close()

Going to download record 1 to 1
Going to download record 2 to 2
Going to download record 3 to 3


In [59]:
dataframe=pd.DataFrame(data_array, columns=['ID', 'date', 'country', 'source'])
dataframe.head()

,ID,date,country,source
0,MW989490,2021-02-27,USA: New Mexico,saliva
1,MW989489,2021-02-27,USA: New Mexico,saliva
2,MW989488,2021-02-25,USA: New Mexico,saliva
